In [ ]:
Embed_Dim = "128"
Head_Num = "8"
Learning_Rate = "0.00001"
LossWeights = "1000"

In [1]:
import os
#drive_path = 'C:/模型 ver. 2/encoder_num=4, decoder_num=4/embed_dim=' + Embed_Dim + '/head_num=' + Head_Num + '/learning_rate=' + Learning_Rate + '/lossWeights_1-' + LossWeights + '/'
drive_path = 'D:/model/'
if not os.path.isdir(drive_path):
    os.makedirs(drive_path)
    
save_path = "D:/save_model/"    
if not os.path.isdir(save_path):
    os.makedirs(save_path)

In [2]:
# encoding: utf-8
from __future__ import unicode_literals
import sys
sys.path.append('D:/local_training')
sys.path.append('D:/local_training/keras_position_wise_feed_forward')
sys.path.append('D:/local_training/tensorflow_fast_attention')
sys.path.append('D:/local_training/keras_performer')

import unittest
import numpy as np
from keras_performer import performerVer2 as tfr
import nltk
from sklearn.model_selection import train_test_split

from os import listdir
from os.path import isfile, isdir, join

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

import re

nltk.download('punkt')
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.optimizers import Adam

import glob

import DataGenerator1 as DG

import DataBuffer as db

from matplotlib import pyplot as plt

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\a2744\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def find_first_sublist(seq, sublist, start=0):
    length = len(sublist)
    for index in range(start, len(seq)):
        if seq[index:index+length] == sublist:
            return index, index+length

def replace_sublist(seq, sublist, replacement):
    length = len(replacement)
    index = 0
    for start, end in iter(lambda: find_first_sublist(seq, sublist, index), None):
        seq[start:end] = replacement
        index = start + length
        
def replaceTAGS(x):
    replace_sublist(x, ['<', 'BOC', '>'], ["<BOC>"])
    replace_sublist(x, ['<', 'EOC', '>'], ["<EOC>"])
    replace_sublist(x, ['<', 'BOTM', '>'], ["<BOTM>"])
    replace_sublist(x, ['<', 'BOT', '>'], ["<BOT>"])
    replace_sublist(x, ['<', 'EOT', '>'], ["<EOT>"])
    replace_sublist(x, ['<', 'BOM', '>'], ["<BOM>"])
    replace_sublist(x, ['<', 'EOM', '>'], ["<EOM>"])
    replace_sublist(x, ['<', 'EOTM', '>'], ["<EOTM>"])
    replace_sublist(x, ['<', 'CR', '>'], ["<CR>"])
    return x

def parseSentence(x):
    tokenizer =  RegexpTokenizer(r"[\w']+|[].,:!?;=+-\\*/@#$%^&_(){}~|\"[]")
    tokens=[]
    state="START"
    chrs=""
    for i in range(len(x)):
        if (ord(x[i])>255):
            inp="U"
        elif (ord(x[i])>=48 and ord(x[i])<=57):
            inp="D"
        else:
            inp="E"

        if state=="START":
            if inp=="D":
                state="NUMBER"
                tokens.append(x[i])
            elif inp=="E":
                state="ASCII"
                chrs=x[i]
            else:#U
                state="UNICODE"
                tokens.append(x[i])                
            
        elif state=="ASCII":	
            if inp=="D" or inp=="E":
                state="ASCII"
                chrs += x[i]
            else:#U
                state="UNICODE"
                tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs) #nltk.word_tokenize(chrs)
                chrs=""
                tokens.append(x[i])

        elif state=="NUMBER":
            if inp=="D":
                state="NUMBER"
                tokens.append(x[i])
            elif inp=="E":
                state="ASCII"
                chrs=x[i]
            else:#U
                state="UNICODE"
                tokens.append(x[i])		

        elif state=="UNICODE":
            if inp=="D":
                state="NUMBER"
                tokens.append(x[i])
            elif inp=="E":
                state="ASCII"
                chrs=x[i]
            else:#U
                state="UNICODE"
                tokens.append(x[i])

    if len(chrs)>0:
        tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs) #nltk.word_tokenize(chrs)
    return replaceTAGS(tokens)

def readcode(fname):
    with open(fname, encoding="utf-8") as f:
        data = f.read()
        return data

def outputsplit(txt): #txt: "<BOTM><BOT>32<EOT><BOM>XXX<EOM><EOTM>...."
    pattern = re.compile(r'<BOTM>(.*?)<EOTM>')
    x = re.findall(r"<BOTM> *<BOT>(.*?)<EOT>(.*?)<EOTM>", txt, re.DOTALL)
    y=list(zip((*x)))#[('32', ...), ('<BOM>XXX<EOM>', ...)]
    err_codes = [int(code) for code in y[0]]
    return err_codes, y[1] #erro int codes, messages

def listdir_fullpath(d):
    return [f for f in os.listdir(d)]
    
def saveMaxLen(filename, data): 
    with open(filename, 'w') as f:
        f.write(str(data))
        f.close()

def saveTestTrainData(filename, data): # e.g., 'test.npy'
    #print(len(data))
    #print(data[0].shape)
    #print(data[1].shape)
    with open(filename, 'wb') as f:
        np.save(f, data)

def saveDictionary(dt, file):
    import pickle
    a_file = open(file, "wb")
    pickle.dump(dt, a_file)
    a_file.close()
        
def loadTestTrainData(filename): # e.g., 'test.npy'    
    with open(filename, 'rb') as f:
        a = np.load(f)
        return a

def loadDictionary(file):
    import pickle
    a_file = open(file, "rb")
    dt = pickle.load(a_file)
    return dt

def load(model_name):
    from keras_performer import performerVer2
    from tensorflow import keras
    from keras_embed_sim import EmbeddingRet, EmbeddingSim
    from keras_pos_embd import TrigPosEmbedding
    from tensorflow_fast_attention.fast_attention import  Attention, SelfAttention
    from keras_position_wise_feed_forward.feed_forward import FeedForward  

    co = performerVer2.get_custom_objects()

    model = keras.models.load_model(model_name, custom_objects= co)
    s = loadDictionary(drive_path + 'source_token_dict.pickle')
    t = loadDictionary(drive_path + 'target_token_dict.pickle')
    t_inv = loadDictionary(drive_path + 'target_token_dict_inv.pickle')
    return model, s, t, t_inv

def saveloss(epochs, train_loss, test_loss):
    path = drive_path + 'loss.txt'
    with open(path, 'a') as f:
        print(epochs, file=f)
        f.write('train_loss\n')
        print("[loss, error_feed_forward_output1_loss, Decoder-Output_loss] =", train_loss, file=f)
        f.write('test_loss\n')
        print("[loss, error_feed_forward_output1_loss, Decoder-Output_loss] =", test_loss, file=f)
        f.write('\n')
        
def loadMaxLen(filename):     
    with open(filename) as f:
        lines = f.readlines()
        return lines
    
def plotTrainingLoss(history):
    plt.plot(history.history['loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.show()

# x, y

In [4]:
class TestTranslate(unittest.TestCase):
    def __init__(self):
        self.source_token_dict = {
            '<PAD>': 0,
            '<START>': 1,
            '<END>': 2,
            '<BOC>': 3,
            '<EOC>': 4,
            '<BOM>': 5,
            '<EOM>': 6,
            '<CR>': 7,
        }
        
        self.target_token_dict = {
            '<PAD>': 0,
            '<START>': 1,
            '<END>': 2,
            '<BOC>': 3,
            '<EOC>': 4,
            '<BOM>': 5,
            '<EOM>': 6,
            '<CR>': 7,
        }

    @staticmethod
    def _build_token_dict(token_dict, token_list):
        for tokens in token_list:
            for token in tokens:
                if token not in token_dict:
                    token_dict[token] = len(token_dict)
        return token_dict

    def test_translate(self):
        Input_Path = "D:/Augmentation/input"
        Output_Path = "D:/Augmentation/output" 
        cases = listdir_fullpath(Input_Path)
        
        source_max_len = 0
        target_max_len = 0
        token_num = 0
        block_num = 120
        for loop in range(0, round(338/block_num)):
            source_tokens = []
            target_errors=[]
            target_tokens = []     
            dirs = block_num if loop < 338//block_num else 338%block_num
            Input_fullpath = []
            Output_fullpath = []
            for i in range(dirs):
                in_path = Input_Path + "/" + cases[loop + i] + "/*.txt"
                Input_fullpath += glob.glob(in_path)
                
            for f in Input_fullpath:
                if isfile(f):
                    source_tokens.append(parseSentence(readcode(f)))
        
            for i in range(dirs):
                out_path = Output_Path + "/" + cases[loop + i] + "/*.txt"
                Output_fullpath += glob.glob(out_path)

            for f in Output_fullpath:
                if isfile(f):  
                    o1, o2 = outputsplit(readcode(f))#o1: list of error codes
                    o2 = "".join(o2)
                    ps = parseSentence(o2) ##<-----parse messages
                    target_errors.append(o1)
                    target_tokens.append(ps)
            
            # Generate dictionaries
            self._build_token_dict(self.source_token_dict, source_tokens)
            self._build_token_dict(self.target_token_dict, target_tokens)
            target_token_dict_inv = {v: k for k, v in self.target_token_dict.items()}

            # Add special tokens
            encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
            decode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in target_tokens]
            output_tokens = [tokens + ['<END>', '<PAD>'] for tokens in target_tokens]
            
            for i in range(len(encode_tokens)):
                if encode_tokens[i] is None or len(encode_tokens[i]) == 0:
                    print("i:", i)
                         
            sl = max(map(len, encode_tokens))
            tl = max(map(len, decode_tokens))
            source_max_len = max(sl, tl, source_max_len)
            saveMaxLen(drive_path + "source_max_len.txt", source_max_len)
            target_max_len = max(sl, tl, target_max_len)
            saveMaxLen(drive_path + "target_max_len.txt", target_max_len)
         
        print("source_max_len:", source_max_len)
        print("target_max_len:", target_max_len)
        #ready to pad and save data
        for loop in range(0, round(338/block_num)):
            print("loop:", loop)
            source_tokens = []
            target_errors=[]
            target_tokens = []     
            dirs = block_num if loop < 338//block_num else 338%block_num
            Input_fullpath = []
            Output_fullpath = []
            for i in range(dirs):
                in_path = Input_Path + "/" + cases[loop + i] + "/*.txt"
                Input_fullpath += glob.glob(in_path)
                
            for f in Input_fullpath:
                if isfile(f):
                    source_tokens.append(parseSentence(readcode(f)))
        
            for i in range(dirs):
                out_path = Output_Path + "/" + cases[loop + i] + "/*.txt"
                Output_fullpath += glob.glob(out_path)

            for f in Output_fullpath:
                if isfile(f):  
                    o1, o2 = outputsplit(readcode(f))#o1: list of error codes
                    o2 = "".join(o2)
                    ps = parseSentence(o2) ##<-----parse messages
                    target_errors.append(o1)
                    target_tokens.append(ps)
                           
            print("XXXX: " , len(source_tokens))
            print("YYYY: " , len(target_errors))
            print("ZZZZ: " , len(target_tokens))

            # Generate dictionaries
            self._build_token_dict(self.source_token_dict, source_tokens)
            self._build_token_dict(self.target_token_dict, target_tokens)
            target_token_dict_inv = {v: k for k, v in self.target_token_dict.items()}

            # Add special tokens
            encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
            decode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in target_tokens]
            output_tokens = [tokens + ['<END>', '<PAD>'] for tokens in target_tokens]
            
            for i in range(len(encode_tokens)):
                if encode_tokens[i] is None or len(encode_tokens[i]) == 0:
                    print("i:", i)
                    
            # Padding
            encode_tokens = [tokens + ['<PAD>'] * (source_max_len - len(tokens)) for tokens in encode_tokens]
            decode_tokens = [tokens + ['<PAD>'] * (target_max_len - len(tokens)) for tokens in decode_tokens]
            output_tokens = [tokens + ['<PAD>'] * (target_max_len - len(tokens)) for tokens in output_tokens]

            encode_input = [list(map(lambda x: self.source_token_dict[x], tokens)) for tokens in encode_tokens]
            decode_input = [list(map(lambda x: self.target_token_dict[x], tokens)) for tokens in decode_tokens]
            decode_output2 = [list(map(lambda x: [self.target_token_dict[x]], tokens)) for tokens in output_tokens]
            
            print("source_token_dict len: ", len(self.source_token_dict))
            print("target_token_dict len: ", len(self.target_token_dict))
            print("target_token_dict_inv len: ", len(target_token_dict_inv))
            
            saveDictionary(self.source_token_dict, drive_path + 'source_token_dict.pickle')
            saveDictionary(self.target_token_dict, drive_path + 'target_token_dict.pickle')
            saveDictionary(target_token_dict_inv, drive_path + 'target_token_dict_inv.pickle')            

            #print("encode_input", np.asarray(encode_input).shape) #(271, 798)
            #print("decode_input", np.asarray(decode_input).shape) #(271, 798)
            #print("decode_output2",  np.asarray(decode_output2).shape) #(271, 798, 1)
            #target errors: into 0/1 arrays from target_errors
            decode_output1 =[ [0]*36 for i in range(len(target_errors))]
            for i in range(len(target_errors)):
                    codes= target_errors[i]
                    for code in codes:  
                            decode_output1[i][code-1] = 1
            #print(decode_output1)


            x=list(zip(np.array(encode_input), np.array(decode_input)))
            y=list(zip(np.array(decode_output1), np.array(decode_output2))) #np.array(decode_output2)

            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

            x_train = list(zip(*x_train))
            x_train[0] = np.array(x_train[0]) #encode_input
            x_train[1] = np.array(x_train[1]) #decode_input
            #print(x_train[0].shape)
            #print(x_train[1].shape)

            y_train = list(zip(*y_train))
            y_train[0] = np.array(y_train[0]) #decode_output1
            y_train[1] = np.array(y_train[1]) #decode_output2
            #print(y_train[0].shape)
            #print(y_train[1].shape)

            x_test = list(zip(*x_test))
            x_test[0] = np.array(x_test[0])
            x_test[1] = np.array(x_test[1])

            y_test = list(zip(*y_test))
            y_test[0] = np.array(y_test[0]) #decode_output1
            y_test[1] = np.array(y_test[1]) #decode_output2
            #print(y_test[0].shape)
            #print(y_test[1].shape)

            #x=[np.array(encode_input * 1), np.array(decode_input * 1)] #(2, 271, 798)
            #y=np.array(decode_output2 * 1) #(271, 798, 1)

            saveTestTrainData(drive_path + "x_train_" + str(loop) + ".npy", x_train)
            saveTestTrainData(drive_path + "x_test_" + str(loop) + ".npy", x_test)
            saveTestTrainData(drive_path + "y_train[0]_" + str(loop) + ".npy", y_train[0])
            saveTestTrainData(drive_path + "y_train[1]_" + str(loop) + ".npy", y_train[1])
            saveTestTrainData(drive_path + "y_test[0]_" + str(loop) + ".npy", y_test[0])
            saveTestTrainData(drive_path + "y_test[1]_" + str(loop) + ".npy", y_test[1])
        
            # x_train, x_test: [array, array ]
            # y_train, y_test: array               

x=TestTranslate()
x.test_translate()

source_max_len: 2769
target_max_len: 2769
loop: 0
XXXX:  120120
YYYY:  120120
ZZZZ:  120120
source_token_dict len:  1291
target_token_dict len:  1132
target_token_dict_inv len:  1132
loop: 1
XXXX:  120120
YYYY:  120120
ZZZZ:  120120
source_token_dict len:  1291
target_token_dict len:  1132
target_token_dict_inv len:  1132
loop: 2
XXXX:  98098
YYYY:  98098
ZZZZ:  98098
source_token_dict len:  1291
target_token_dict len:  1132
target_token_dict_inv len:  1132


# Training

In [ ]:
class TestTranslate(unittest.TestCase):

    def test_translate(self):

        source_token_dict = loadDictionary(drive_path + 'source_token_dict.pickle')
        target_token_dict = loadDictionary(drive_path + 'target_token_dict.pickle')
        target_token_dict_inv = loadDictionary(drive_path + 'target_token_dict_inv.pickle')
        
        print("source_token_dict len: ", len(source_token_dict))
        print("target_token_dict len: ", len(target_token_dict))
        print("target_token_dict_inv len: ", len(target_token_dict_inv))
        
        source_max_len_loaded = loadMaxLen(drive_path + "source_max_len.txt")
        source_max_len = int(source_max_len_loaded[0])
        
        target_max_len_loaded = loadMaxLen(drive_path + "target_max_len.txt")
        target_max_len = int(target_max_len_loaded[0])

        # Build & fit model      
        model = tfr.get_model(
            max_input_len=(source_max_len, target_max_len),
            token_num=max(len(source_token_dict), len(target_token_dict)),
            embed_dim=32,
            encoder_num=2,
            decoder_num=2,
            head_num=4,
            hidden_dim=128,
            dropout_rate=0.05,
            use_same_embed=False  # Use different embeddings for different languages
        )

        losses = {
                "error_feed_forward_output1": "binary_crossentropy",
                "Decoder-Output": "sparse_categorical_crossentropy",
        }
        lossWeights = {"error_feed_forward_output1": 1.0, "Decoder-Output": 1000.0}
        model.compile(optimizer=Adam(learning_rate=0.00001), loss=losses, loss_weights=lossWeights) 
        model.summary()
        
        #for output 
        output_buffer_params = { 
            "data_path": ["D:\\model\\", "D:\\model\\"],
            "data_number":[270670, 270670],
            "data_type": [int, int],
            "block_size": [96096, 96096]
            }
        #for input
        input_buffer_params = {
            "data_path": ["D:\\model\\", "D:\\model\\"],
            "data_number":[270670, 270670],
            "data_type": [int, int],
            "block_size": [96096, 96096]
            }
        
        # Generators
        print("call DataGenerator1...")
        training_generator = DG.DataGenerator1(input_buffer_params, output_buffer_params,
                                               [list(range(270670)), list(range(270670))]
                                              )
        
        #validation_generator = DataGenerator(partition['validation'], labels, **params)
        
        # Train model on dataset
        history = model.fit_generator(
            generator=training_generator,        
            epochs=2,      
        )
        
        plotTrainingLoss(history)
        
        model.save(save_path + "test_model_2.h5")   
        print("model save end...")
        
        print(" ")
        print("train_loss")
        train_loss = model.evaluate(x_train, y_train)
        print("test_loss")
        test_loss = model.evaluate(x_test, y_test) 
        saveloss(2, train_loss, test_loss)

x=TestTranslate()
x.test_translate()

source_token_dict len:  1291
target_token_dict len:  1132
target_token_dict_inv len:  1132
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder-Input (InputLayer)      [(None, 2769)]       0                                            
__________________________________________________________________________________________________
Encoder-Token-Embedding (Embedd [(None, 2769, 32), ( 41312       Encoder-Input[0][0]              
__________________________________________________________________________________________________
Encoder-Embedding (TrigPosEmbed (None, 2769, 32)     0           Encoder-Token-Embedding[0][0]    
__________________________________________________________________________________________________
self_attention_1 (SelfAttention (None, 2769, 32)     4096        Encoder-Embedding[0][0]          
___

C:\Users\a2744\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
